In [ ]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# dataset_directory = "../Pubmed"
pubmed_dataset = Planetoid(root='', name='Pubmed')
data = pubmed_dataset[0].to(device)
print(data)

trials = 5

In [ ]:
edges_to_add = None

In [ ]:
model = GCN(data.x.shape[1], pubmed_dataset.num_classes, [16]).to(device)

In [ ]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/pubmed_gcn_edges_full_run.pth"
# list_save_path = "../attacks/pubmed_gcn_edges.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [ ]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [ ]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

In [ ]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [ ]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_greedy(data, train, model, 0.0, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

In [ ]:
edges_to_add

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

# regular, secondary attack, not reversed

In [ ]:
# split_dic_acc

In [ ]:
splits = [0.0, 0.5]

split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [ ]:
# data

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_acc = dict(split_dic_acc)
split_dic_loss = dict(split_dic_loss)
itrs_one = dict(itrs_one)

In [ ]:
record1 = Recording(split_dic_loss, split_dic_acc, itrs_one, Model.GCN, Dataset.PUBMED, AcceptFn.CONSTANT, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"pubmed"}_{"constant"}_{"attacked"}_results.pkl')

In [ ]:
split_dic_loss

In [ ]:
# plot_results({0: split_dic_acc[0.0][0], 0.5: split_dic_acc[0.5][0]}, ptb_rate, "Greedy", "", "constant", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_loss[0.0][0], 0.5: split_dic_loss[0.5][0]}, ptb_rate, "Greedy", "", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [ ]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_decreasing_acc = dict(split_dic_decreasing_acc)
split_dic_decreasing_loss = dict(split_dic_decreasing_loss)
itrs_two = dict(itrs_two)

In [ ]:
record1 = Recording(split_dic_decreasing_loss, split_dic_decreasing_acc, itrs_two, Model.GCN, Dataset.PUBMED, AcceptFn.INCREASING, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"pubmed"}_{"increasing"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results({0: split_dic_decreasing_acc[0.0][0], 0.5: split_dic_decreasing_acc[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_decreasing_loss[0.0][0], 0.5: split_dic_decreasing_loss[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "loss", "regular")